# K-means 군집화 알고리즘
 - step1. 데이터들을 임의로 k개의 군집에 할당하고, 각각 같은 군집에 속한 객체들의 중심점을 구한다.
 - step2. 구해진 중심점들과 객체사이의 거리를 구해 가까운 쪽으로 데이터를 재할당한다.
 - step3. 재할당된 상태에서 다시 각각 같은 군집에 속한 객체들의 중심점을 구한다.
 - step4. 더이상 군집이 바뀌지 않을 때 까지 setp2~3를 반복한다.
 - 각 군집 중심점의 초기값을 랜덤하게 정하기 때문에 초기값 위치에 따라 원하는 결과가 나오지 않을 수 있다.(지역해)
 - 군집의 크기가 다를 경우 제대로 작동되지 않을 수 있고, 밀도가 다를 경우에도 마찬가지다.
 - k값이 많아지면 클러스터링이 잘 되는 결과를 도출하지만 너무 많아지면 클러스터링의 의미가 없어진다.
 - 중심점을 구하는 과정에서 이상치가 있으면 중심점이 왜곡될 수 있다.
 - 본 코드는 k를 3개라고 가정한다.
 - 유클리디안 거리측도를 활용한 cost에 추가적으로 Silhouette 평가 값도 구해본다.

In [48]:
import pandas as pd
import numpy as np
import random
import copy

In [49]:
df=pd.read_csv("C:/Users/SOP/data/wine.csv")

In [50]:
df

,Alcohol,Malic.acid,Ash,Acl,Mg,Phenols,Flavanoids,Nonflavanoid.phenols,Proanth,Color.int,Hue,OD,Proline
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.640000,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.380000,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.680000,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.800000,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.320000,1.04,2.93,735
5,14.20,1.76,2.45,15.2,112,3.27,3.39,0.34,1.97,6.750000,1.05,2.85,1450
6,14.39,1.87,2.45,14.6,96,2.50,2.52,0.30,1.98,5.250000,1.02,3.58,1290
7,14.06,2.15,2.61,17.6,121,2.60,2.51,0.31,1.25,5.050000,1.06,3.58,1295
8,14.83,1.64,2.17,14.0,97,2.80,2.98,0.29,1.98,5.200000,1.08,2.85,1045
9,13.86,1.35,2.27,16.0,98,2.98,3.15,0.22,1.85,7.220000,1.01,3.55,1045


In [51]:
# step1. 데이터들을 k개의 군집에 할당한다. (k=3)

for i in range(178):
    df.loc[i, 'cluster'] = np.random.randint(0,3)

In [52]:
# 13개의 attribute와 3개의 k에 대한 중심점을 표현한다.

center = {}
for i in range(3):
    center[i] = [0,0,0,0,0,0,0,0,0,0,0,0,0]

In [53]:
# 중심점을 구하는 함수이다.

def update(a):
    for i in range(3):
        center[i][0]=sum(df[df['cluster']==i]['Alcohol']/len(list(df[df['cluster']==i]['Alcohol'])))
        center[i][1]=sum(df[df['cluster']==i]['Malic.acid']/len(list(df[df['cluster']==i]['Malic.acid'])))
        center[i][2]=sum(df[df['cluster']==i]['Ash']/len(list(df[df['cluster']==i]['Ash'])))
        center[i][3]=sum(df[df['cluster']==i]['Acl']/len(list(df[df['cluster']==i]['Acl'])))
        center[i][4]=sum(df[df['cluster']==i]['Mg']/len(list(df[df['cluster']==i]['Mg'])))
        center[i][5]=sum(df[df['cluster']==i]['Phenols']/len(list(df[df['cluster']==i]['Phenols'])))
        center[i][6]=sum(df[df['cluster']==i]['Flavanoids']/len(list(df[df['cluster']==i]['Flavanoids'])))
        center[i][7]=sum(df[df['cluster']==i]['Nonflavanoid.phenols']/len(list(df[df['cluster']==i]['Nonflavanoid.phenols'])))
        center[i][8]=sum(df[df['cluster']==i]['Proanth']/len(list(df[df['cluster']==i]['Proanth'])))
        center[i][9]=sum(df[df['cluster']==i]['Color.int']/len(list(df[df['cluster']==i]['Color.int'])))
        center[i][10]=sum(df[df['cluster']==i]['Hue']/len(list(df[df['cluster']==i]['Hue'])))
        center[i][11]=sum(df[df['cluster']==i]['OD']/len(list(df[df['cluster']==i]['OD'])))
        center[i][12]=sum(df[df['cluster']==i]['Proline']/len(list(df[df['cluster']==i]['Proline'])))
    return a

In [54]:
center = update(center)

In [55]:
center

{0: [13.064107142857138,
  2.274642857142857,
  2.389107142857144,
  19.782142857142848,
  101.73214285714286,
  2.353214285714286,
  2.086428571428572,
  0.36928571428571433,
  1.5453571428571429,
  5.265535714285715,
  0.9651071428571429,
  2.7301785714285707,
  784.8214285714286],
 1: [12.88565217391304,
  2.2885507246376813,
  2.372028985507247,
  19.68115942028986,
  97.79710144927536,
  2.289999999999999,
  2.0239130434782604,
  0.37420289855072475,
  1.6908695652173917,
  4.8447825942029,
  0.9676811594202899,
  2.5921739130434776,
  730.5507246376811],
 2: [13.083207547169813,
  2.4637735849056597,
  2.335471698113208,
  18.94905660377359,
  100.16981132075475,
  2.240377358490566,
  1.975849056603774,
  0.33792452830188674,
  1.5088679245283023,
  5.116603773584909,
  0.9360377358490566,
  2.511886792452829,
  728.0943396226417]}

In [56]:
# step2.  구해진 중심점들과 객체사이의 거리를 구해 가까운 쪽으로 데이터를 재할당하는 함수이다.

def assignment(df, center):
    for i in range(3):
        df['distance_from_{}'.format(i)]=(
            np.sqrt(
                (df.iloc[:,0]-center[i][0])**2
                +(df.iloc[:,1]-center[i][1])**2
                +(df.iloc[:,2]-center[i][2])**2
                +(df.iloc[:,3]-center[i][3])**2
                +(df.iloc[:,4]-center[i][4])**2
                +(df.iloc[:,5]-center[i][5])**2
                +(df.iloc[:,6]-center[i][6])**2
                +(df.iloc[:,7]-center[i][7])**2
                +(df.iloc[:,8]-center[i][8])**2
                +(df.iloc[:,9]-center[i][9])**2
                +(df.iloc[:,10]-center[i][10])**2
                +(df.iloc[:,11]-center[i][11])**2
                +(df.iloc[:,12]-center[i][12])**2
            )
        )       
    distance_from_all = ['distance_from_{}'.format(i) for i in range(3)]
    df['cluster'] = df.loc[:,distance_from_all].idxmin(axis=1)
    df['cluster'] = df['cluster'].map(lambda x: int(x.lstrip('distance_from_')))
    return df

In [57]:
df = assignment(df,center)

In [58]:
# step4. 더이상 군집이 바뀌지 않을 때 까지 setp2~3를 반복한다.

while True: # 무한 루프를 통해
    original_min_distance_center = df['cluster'].copy(deep=True) # 원래 클러스터를 저장해둔다.
    center = update(center)
    df=assignment(df, center)
    if original_min_distance_center.equals(df['cluster']): # 클러스터가 바뀌지 않으면 루프를 종료한다.
        break

In [59]:
df

,Alcohol,Malic.acid,Ash,Acl,Mg,Phenols,Flavanoids,Nonflavanoid.phenols,Proanth,Color.int,Hue,OD,Proline,cluster,distance_from_0,distance_from_1,distance_from_2
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.640000,1.04,3.92,1065,0,131.922633,337.512945,607.788267
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.380000,1.05,3.40,1050,0,145.379821,321.806719,591.898618
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.680000,1.03,3.17,1185,0,11.287394,456.676785,726.828633
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.800000,0.86,3.45,1480,0,284.960393,751.736649,1021.997058
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.320000,1.04,2.93,735,1,460.338931,16.040100,277.959202
5,14.20,1.76,2.45,15.2,112,3.27,3.39,0.34,1.97,6.750000,1.05,2.85,1450,0,254.943427,721.731319,991.986304
6,14.39,1.87,2.45,14.6,96,2.50,2.52,0.30,1.98,5.250000,1.02,3.58,1290,0,95.363528,561.742428,831.803935
7,14.06,2.15,2.61,17.6,121,2.60,2.51,0.31,1.25,5.050000,1.06,3.58,1295,0,101.054756,566.937044,837.268081
8,14.83,1.64,2.17,14.0,97,2.80,2.98,0.29,1.98,5.200000,1.08,2.85,1045,0,150.425345,316.796679,586.834573
9,13.86,1.35,2.27,16.0,98,2.98,3.15,0.22,1.85,7.220000,1.01,3.55,1045,0,150.349621,316.749563,586.829651


In [60]:
# 유클리디안 거리측도(평가값) 구하기

distance_min=[]
for i in range(178):
    distance_min.append(min(list(df.iloc[i,14:17])))

In [61]:
value = sum(distance_min)

In [62]:
print(value)

16555.67941603272


## Silhouette
- 실루엣 값은 클러스터의 성능을 측정할 때 사용하는 지표이다.
- 한 클러스터 안의 데이터들이 다른 클러스터와 비교해서 얼마나 비슷한가를 나타낸다.
- 클러스터 안의 거리가 짧을 수록 좋고(cohesion), 다른 클러스터와의 거리는 멀수록 좋다(separation)
- 실루엣은 -1부터 1사이의 값을 가진다. 값이 높을 수록 성능이 좋다고 할 수 있다
- a(i) : 같은 클러스터 안에 있는 다른 데이터들과의 평균거리. a(i)는 i번째 데이터가 클러스터에 얼마나 잘 맞는지를 측정한다.(낮을 수록 현재 클러스터에 더 잘 속해있는 것)
- b(i) : i가 속하지 않은 다른 클러스터와의 평균 거리 중 가장 작은 거리이다. (클 수록 현재 클러스터에 더 잘 속해있는 것

In [63]:
#silhouette 평가 값 산출과정

for i in range(178): #i번재 데이터를 기준으로
    for j in range(178): #모든 데이터들과 거리를 구한다.
            z=np.sqrt(
            (df.iloc[i,0]-df.iloc[j,0])**2
            +(df.iloc[i,1]-df.iloc[j,1])**2
            +(df.iloc[i,2]-df.iloc[j,2])**2
            +(df.iloc[i,3]-df.iloc[j,3])**2
            +(df.iloc[i,4]-df.iloc[j,4])**2
            +(df.iloc[i,5]-df.iloc[j,5])**2
            +(df.iloc[i,6]-df.iloc[j,6])**2
            +(df.iloc[i,7]-df.iloc[j,7])**2
            +(df.iloc[i,8]-df.iloc[j,8])**2
            +(df.iloc[i,9]-df.iloc[j,9])**2
            +(df.iloc[i,10]-df.iloc[j,10])**2
            +(df.iloc[i,11]-df.iloc[j,11])**2
            +(df.iloc[i,12]-df.iloc[j,12])**2
            +(df.iloc[i,13]-df.iloc[j,13])**2)
            df.loc[j,'distance_from_data{}'.format(i)]=z


In [64]:
#각 데이터들마다 자신이 속한 Cluster의 거리를 x리스트에 나타냄

x=[]
for i in range(178): 
    a=df.loc[i,'cluster']
    dis=[]
    for j in range(178): #모든 데이터들에 대해
            if df.loc[j,'cluster']==a: #같은 cluster인 경우.
                dis.append(df.iloc[j,8+i]) #dis리스트에 거리 값 추가(8번째 열이 distance_from_data0)
    x.append(sum(dis)) #거리 값들을 더한 값을 x리스트에 추가

In [65]:
x

[89.79,
 268.02000000000004,
 50.67999999999999,
 146.36,
 45157,
 0,
 6676.1505172894995,
 21949.437601772974,
 34646.35678577033,
 7521.734326933903,
 7682.4726411289275,
 6632.8467793086065,
 14070.089544400373,
 21645.604437626444,
 12959.242091001466,
 7714.706230969665,
 7901.727620459796,
 7806.456801666365,
 7801.17787365201,
 48475.52752834334,
 34227.18561123908,
 36713.77178780431,
 6648.210763411032,
 16817.103020485032,
 36083.352877223646,
 34234.432051404285,
 6686.002306730463,
 22795.086494867388,
 8227.369555638183,
 19549.590928450078,
 19993.06699611109,
 8083.005156786514,
 8799.31674245201,
 16479.243122342436,
 17205.909249946846,
 28966.171302610637,
 34538.07105960652,
 13189.029662941666,
 8096.456106599565,
 34532.22601065483,
 48785.402337916494,
 9760.498984184405,
 7118.99784784329,
 22764.71625524827,
 12014.251268499771,
 14830.367003282832,
 6830.391565461374,
 8601.788042846227,
 20493.778012685256,
 18826.73837351604,
 8151.450354068845,
 6915.5896751

In [66]:
#각 데이터들마다 자신이 속하지 않은 cluster의 거리를 y리스트에 나타냄

y=[]
for i in range(178): #i는 몇 번째 데이터인지
    other_cluster={0:0,1:0,2:0} #복수개의 cluster와의 거리를 나타내기위해 딕셔너리자료형{key:Value} 선언
    a=df.loc[i,'cluster']
    dis=[]
    for j in range(178):
            if df.loc[j,'cluster']!=a: #속하지않은 cluster의 경우
                #딕셔너리[key]=Value 형태를 사용. 반복적으로 그 값을 더해나간다.
                other_cluster[df.loc[j,'cluster']]=other_cluster[df.loc[j,'cluster']]+df.iloc[j,8+i]
    #3개의 cluster를 딕셔너리로 선언했기에 만약 자신의 클러스터일경우를 대비 (min값에 영향)
    if other_cluster[0]== 0:
        other_cluster[0]=999999
    elif other_cluster[1]==0:
        other_cluster[1]=999999
    elif other_cluster[2]==0:
        other_cluster[2]=999999        
    y.append(min(other_cluster[0],other_cluster[1],other_cluster[2]))

In [67]:
silhouette_list=[]
for i in range(178):
    sil=(y[i]-x[i])/max(x[i],y[i])
    silhouette_list.append(sil)
sum(silhouette_list)/178

0.025708397757452314